# CEOS Open Data Cube Notebooks

The CEOS Data Cube Notebooks is a collection of [Jupyter Notebooks](https://jupyter.org/) using the [Open Data Cube](https://www.opendatacube.org/) to analyze satellite data.

For an overview of the available notebooks, see the [Notebooks Overview](docs/notebooks_overview.md) document.

## Installation
-------

First follow the instructions in the [Docker Installation Guide](https://github.com/ceos-seo/data_cube_ui/blob/master/docs/docker_install.md) if you do not have Docker installed yet.

Follow the instructions in the 
[Open Data Cube Database Installation Guide](https://github.com/ceos-seo/data_cube_ui/blob/master/docs/odc_db_setup.md) to setup the Open Data Cube (ODC) database.

Follow the instructions in the [Operation Manual](docs/notebooks_operation_manual.md) to set up and operate the Jupyter Notebook environment.

## Obtaining Help
-------

If you encounter issues with the Open Data Cube or these notebooks that are not documented in the documents linked to above or the other files in the `docs` directory (remember to check the FAQ sections) and you are unable to diagnose and fix those issues on your own, follow these steps to obtain assistance:
1. Search for your question in the [GIS Stack Exchange](https://gis.stackexchange.com/) with the `open-data-cube` tag. You can find such tagged questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube). If a question similar to yours has already been asked, search for a suitable answer in that question's webpage. If you can not find a suitable answer, contine to step 2. If no such question exists or if the question is missing some information that you think may be important regarding your issue, create a new question with the `open-data-cube` tag.
2. State your problem in the [ODC Slack workspace](http://slack.opendatacube.org/) in the most appropriate channel (use that link to join the workspace if you have not already). Use the **#general** channel if no other channel is more appropriate for your question. Link to the question on GIS Stack Exchange. When you receive an answer, add that answer to the question page on GIS Stack Exchange if the answerer is not available to do so. This preserves the answer in a publicly searchable way - which is useful for remembering answers to one's own past questions and benefits the community.

## More
-------

You may also consider running a webapp that uses the Open Data Cube. The CEOS ODC UI repository is [here](https://github.com/ceos-seo/data_cube_ui).